# pycos: Framework for concurrent & distributed Python programming

## Basic Task Creation

The main abstraction to keep in mind with pycos is that it uses generator functions to create its tasks:

In [1]:
import time
import random
import pycos

def task_proc(n, task=None):
    delay = random.uniform(0.5, 3)
    print(f'{time.ctime()}: task {n} sleeping for {delay}s')
    yield task.sleep(delay)
    print(f'{time.ctime()}: task {n} terminating')
    
for i in range(10):
    pycos.Task(task_proc, i)
    
pycos.Pycos().join()

2020-05-22 14:13:32 pycos - version 4.8.15 with kqueue I/O notifier
Fri May 22 14:13:32 2020: task 0 sleeping for 2.950675395980922s
Fri May 22 14:13:32 2020: task 2 sleeping for 1.225569924855368s
Fri May 22 14:13:32 2020: task 3 sleeping for 2.089571592356525s
Fri May 22 14:13:32 2020: task 4 sleeping for 1.360957337504992s
Fri May 22 14:13:32 2020: task 1 sleeping for 0.9288766549316738s
Fri May 22 14:13:32 2020: task 5 sleeping for 2.775895242900838s
Fri May 22 14:13:32 2020: task 6 sleeping for 1.390637964854669s
Fri May 22 14:13:32 2020: task 7 sleeping for 1.9447317248872822s
Fri May 22 14:13:32 2020: task 8 sleeping for 1.9480234434917894s
Fri May 22 14:13:32 2020: task 9 sleeping for 1.0908319337399388s
Fri May 22 14:13:33 2020: task 1 terminating
Fri May 22 14:13:33 2020: task 9 terminating
Fri May 22 14:13:33 2020: task 2 terminating
Fri May 22 14:13:33 2020: task 4 terminating
Fri May 22 14:13:33 2020: task 6 terminating
Fri May 22 14:13:34 2020: task 8 terminating
Fri May 

## Sending messages between tasks

Tasks have a `send()` and `receive()` method for handling communications:

In [2]:
n_producer = 3
msgs_per_producer = 3

def consumer_proc(task=None):
    task.set_daemon()
    while True:
        message = yield task.receive()
        print(f'Received {message}')
            
def producer_proc(consumer, i, task=None):
    for j in range(msgs_per_producer):
        yield task.sleep(random.uniform(0.5, 3))
        msg = f'Message {i}-{j}'
        consumer.send(msg)
    print(f'Producer {i} exiting')
        
            
consumer = pycos.Task(consumer_proc)
for i in range(n_producer):
    pycos.Task(producer_proc, consumer, i)
    
pycos.Pycos().join()

Received Message 0-0
Received Message 1-0
Received Message 2-0
Received Message 1-1
Received Message 0-1
Received Message 2-1
Producer 2 exiting
Received Message 2-2
Producer 1 exiting
Received Message 1-2
Producer 0 exiting
Received Message 0-2


## Pub/Sub with channels

If you have _multiple_ consumers of a single message, you can use a `pycos.Channel` to provide a pub/sub architecture:

In [3]:
n_consumer = 3
n_message = 3

def consumer_proc(n, task=None):
    while True:
        msg = yield task.receive()
        if msg is None:
            break
        print(f'Consumer {n} received {msg}')
    print(f'Exiting consumer {n}')
    

def producer_proc():
    channel = pycos.Channel('somename')
    consumers = [
        pycos.Task(consumer_proc, i)
        for i in range(n_consumer)
    ]
    for c in consumers:
        yield channel.subscribe(c)
    for i in range(n_message):
        channel.send(f'Message {i}')
    channel.send(None)
    for c in consumers:
        yield channel.unsubscribe(c)
        
pycos.Task(producer_proc)
pycos.Pycos().join()

Consumer 1 received Message 0
Consumer 2 received Message 0
Consumer 0 received Message 0
Consumer 1 received Message 1
Consumer 2 received Message 1
Consumer 0 received Message 1
Consumer 1 received Message 2
Consumer 2 received Message 2
Consumer 0 received Message 2
Exiting consumer 1
Exiting consumer 2
Exiting consumer 0


## Socket support

We can also do basic socket programming with the `pycos.AsyncSocket()` wrapper:

In [4]:
import socket

def handler_proc(sock, task=None):
    while True:
        data = yield sock.recv(1024)
        print(f'Task {task} Received {data}')
        if not data:
            break
        yield sock.sendall(data)
    sock.close()
        
def server_proc(sock, host, port, task=None):
    task.set_daemon()
    sock = pycos.AsyncSocket(sock)
    sock.bind((host, port))
    sock.listen()

    while True:
        conn, addr = yield sock.accept()
        pycos.Task(handler_proc, conn)
        
sock = socket.socket()
server = pycos.Task(server_proc, sock, '127.0.0.1', 8200)
server

!server_proc/4428999760

Task !handler_proc/4428998416 Received b'Here\n'
Task !handler_proc/4428998416 Received b'is\n'
Task !handler_proc/4428998416 Received b'data\n'
Task !handler_proc/4428998608 Received b"Here's \n"
Task !handler_proc/4428998608 Received b'another\n'
Task !handler_proc/4428998608 Received b'task\n'
Task !handler_proc/4428998416 Received b''
Task !handler_proc/4428998608 Received b''


In [5]:
sock.close()
server.terminate()

0

## Distributed programming

Tasks can send/receive messages even if they are not on the same machine. In order for this to work, we'll need to import `netpycos`. 

In [6]:
!ifconfig | grep inet

	inet 127.0.0.1 netmask 0xff000000 
	inet6 ::1 prefixlen 128 
	inet6 fe80::1%lo0 prefixlen 64 scopeid 0x1 
	inet 192.168.1.70 netmask 0xffffff00 broadcast 192.168.1.255
	inet6 fe80::18da:a5ff:feee:93a8%awdl0 prefixlen 64 scopeid 0xa 
	inet6 fe80::8fb0:d7df:7ca4:f7a0%utun0 prefixlen 64 scopeid 0xb 


In [7]:
!echo "MY_IP = '192.168.1.70'" > data/pycos-examples/my_ip.py

In [8]:
%%file data/pycos-examples/receiver.py
import random, sys
import pycos
import pycos.netpycos

from my_ip import MY_IP

def server_proc(task=None):
    task.set_daemon()
    task.register('receiver_task')
    while True:
        msg = yield task.receive()
        print('received %s' % (msg))

sched = pycos.Pycos.instance(node=MY_IP)
#sched = pycos.Pycos.instance()
server = pycos.Task(server_proc)
while True:
    cmd = sys.stdin.readline().strip().lower()
    if cmd == 'quit' or cmd == 'exit':
        break

Overwriting data/pycos-examples/receiver.py


In [9]:
%%file data/pycos-examples/producer.py
import random
import pycos
import pycos.netpycos

from my_ip import MY_IP

n_producer = 3
msgs_per_producer = 3
# sched = pycos.Pycos.instance()
sched = pycos.Pycos.instance(node=MY_IP)

def main_proc(task=None):
    print('Entering main_proc')
    consumer = yield task.locate('receiver_task')
    print(f'Located {consumer!r}, type {type(consumer)}')
    for i in range(n_producer):
        pycos.Task(producer_proc, consumer, i)


def producer_proc(consumer, i, task=None):
    print('Entering producer_proc')
    for j in range(msgs_per_producer):
        yield task.sleep(random.uniform(0.5, 3))
        msg = f'Message {i}-{j}'
        consumer.send(msg)
    print(f'Producer {i} exiting')
    
pycos.Task(main_proc)

pycos.Pycos().join()

Overwriting data/pycos-examples/producer.py


## Distributed channels

We can also use distributed *channels* for pub/sub communications:

In [10]:
%%file data/pycos-examples/counter.py
import sys
import pycos
import pycos.netpycos

from my_ip import MY_IP

sched = pycos.Pycos.instance(node=MY_IP)
# sched = pycos.Pycos.instance()
channel = pycos.Channel('counter')

def server_proc(task=None):
    task.set_daemon()
    channel.register()
    i = 0
    while True:
        channel.send(i)
        yield task.sleep(1)
        i += 1

server = pycos.Task(server_proc)
while True:
    cmd = sys.stdin.readline().strip().lower()
    if cmd == 'quit' or cmd == 'exit':
        break

Overwriting data/pycos-examples/counter.py


In [11]:
%%file data/pycos-examples/listener.py
import sys
import pycos
import pycos.netpycos

from my_ip import MY_IP

sched = pycos.Pycos.instance(node=MY_IP)
# sched = pycos.Pycos.instance()

def main_proc(task=None):
    task.set_daemon()
    chan = yield pycos.Channel.locate('counter')
    print(f'Located {chan}')
    yield chan.subscribe(task)
    while True:
        msg = yield task.receive()
        print(f'received {msg}')
    
pycos.Task(main_proc)
while True:
    cmd = sys.stdin.readline().strip().lower()
    if cmd == 'quit' or cmd == 'exit':
        break


Overwriting data/pycos-examples/listener.py


## Distributed and communicating tasks

With the `pycos.dispycos` package and some extra work, we can actually send the Python function to the remote node to be executed (without registering it first).

The following code is adapted from example code distributed with pycos:

(You must start the dispycosnode.py program before running this script)

In [14]:
%%file data/pycos-examples/dispycos_client1.py
# Run 'dispycosnode.py' program to start processes to execute computations sent
# by this client, along with this program.

# Distributed computing example where this client sends computation ('compute'
# function) to remote dispycos servers to run as remote tasks and obtain
# results. At any time at most one computation task is scheduled at a process,
# as the computation is supposed to be CPU heavy (although in this example they
# are not).

import pycos
import pycos.netpycos
from pycos.dispycos import *
from my_ip import MY_IP


# this generator function is sent to remote dispycos servers to run tasks there
def compute(i, n, task=None):
    import time
    yield task.sleep(n)
    return((i, task.location, time.asctime()))  # result of 'compute' is current time


# client (local) task submits computations
def client_proc(computation, njobs, task=None):
    # schedule computation with the scheduler; scheduler accepts one computation
    # at a time, so if scheduler is shared, the computation is queued until it
    # is done with already scheduled computations
    if (yield computation.schedule()):
        raise Exception('Could not schedule computation')

    # arguments must correspond to arguments for computaiton; multiple arguments
    # (as in this case) can be given as tuples
    args = [(i, random.uniform(2, 5)) for i in range(njobs)]
    results = yield computation.run_results(compute, args)
    # Tasks may not be executed in the order of given list of args, but
    # results would be in the same order of given list of args
    for result in results:
        print('    result for %d from %s: %s' % result)

    # wait for all jobs to be done and close computation
    yield computation.close()


if __name__ == '__main__':
    import sys, random
    pycos.logger.setLevel(pycos.Logger.DEBUG)
    # PyPI / pip packaging adjusts assertion below for Python 3.7+
    if sys.version_info.major == 3:
        assert sys.version_info.minor >= 7, \
            ('"%s" is not suitable for Python version %s.%s; use file installed by pip instead' %
             (__file__, sys.version_info.major, sys.version_info.minor))

    # if scheduler is not already running (on a node as a program), start
    # private scheduler:
    Scheduler(node=MY_IP)
    # Scheduler()
    # package computation fragments
    computation = Computation([compute])
    # run 10 (or given number of) jobs
    pycos.Task(client_proc, computation, 10 if len(sys.argv) < 2 else int(sys.argv[1]))


Overwriting data/pycos-examples/dispycos_client1.py


# Lab 

Open [pycos lab](./pycos-lab.ipynb)